In [ ]:
# pip install with locked versions
! pip install -U pip
! pip install -U clearml==0.16.2rc0

In [ ]:
from clearml import Task
from clearml.automation.controller import PipelineController

In [ ]:
task = Task.init(project_name='Tabular Example', task_name='tabular training pipeline', task_type=Task.TaskTypes.controller)

In [ ]:
pipe = PipelineController(default_execution_queue='dan_queue', add_pipeline_tags=True)
pipe.add_step(name='preprocessing_1', base_task_project='Tabular Example', base_task_name='tabular preprocessing',
              parameter_override={'General/data_task_id': '39fbf86fc4a341359ac6df4aa70ff91b',
                                  'General/fill_categorical_NA': 'True',
                                  'General/fill_numerical_NA': 'True'})
pipe.add_step(name='preprocessing_2', base_task_project='Tabular Example', base_task_name='tabular preprocessing',
              parameter_override={'General/data_task_id': '39fbf86fc4a341359ac6df4aa70ff91b',
                                  'General/fill_categorical_NA': 'False',
                                  'General/fill_numerical_NA': 'True'})
                                  
pipe.add_step(name='train_1', parents=['preprocessing_1'],
              base_task_project='Tabular Example', base_task_name='tabular prediction',
              parameter_override={'General/data_task_id': '${preprocessing_1.id}'})
pipe.add_step(name='train_2', parents=['preprocessing_2'],
              base_task_project='Tabular Example', base_task_name='tabular prediction',
              parameter_override={'General/data_task_id': '${preprocessing_2.id}'})
                                  
pipe.add_step(name='pick_best', parents=['train_1', 'train_2'],
              base_task_project='Tabular Example', base_task_name='pick best model',
              parameter_override={'General/train_tasks_ids': '[${train_1.id}, ${train_2.id}]'})                   

In [ ]:
# Starting the pipeline (in the background)
pipe.start()
# Wait until pipeline terminates
pipe.wait()
# cleanup everything
pipe.stop()